### Author: Hoang Chu

### Imports

In [ ]:
%pip install -r dependencies.txt

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import requests
import scipy
import importlib
import helper, consts
importlib.reload(consts)
importlib.reload(helper)
import glob

### CONSTANTS

In [2]:
ROW = consts.ROW
COL = consts.COL
RAW_DATA_PATH = consts.RAW_DATA_PATH
RESPONSE_NAME = consts.RESPONSE_NAME

RESPONSE_NAME

'tonight'

### Quick Functions

In [ ]:
does_sub_df_has_Nan = lambda df, col_name: df[[col_name]].isna().any(axis=COL)

### Read data

In [ ]:
trainRange = helper.get_train_from_testday('20150102')
files = helper.get_file_names(trainRange[0],trainRange[1]) # get filenames from before 6/1/2015

In [ ]:
# local read

# take in testing day - split into year and month 



files = helper.get_file_names('20150101','20150601')
dfs = [pd.read_csv(consts.DATA_PATH_2015 + f"{f}") for f in files]
main_df = pd.concat(dfs)
main_df.shape
test_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150601_1200.csv")
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]
saved_cols = x_cols + [RESPONSE_NAME]
training_df = main_df[saved_cols]
testing_df = test_df[saved_cols]

# folder_path = '../data/training_data'
# file_list = glob.glob(folder_path + "/*.csv") 
# main_df = pd.DataFrame(pd.read_csv(file_list[0])) 
# for i in range(1,len(file_list)): 
#     df = pd.read_csv(file_list[i]) 
#     # df = pd.DataFrame(data) 
#     main_df = pd.concat([main_df,df],axis=0) 

# df = pd.concat(map(pd.read_csv,glob.glob(folder_path + '/*.csv')))

#data_20191202_1200_df = pd.read_csv(RAW_DATA_PATH + "data_20191202_1200.csv")
#data_20191203_1200_df = pd.read_csv(RAW_DATA_PATH + "data_20191203_1200.csv")

In [ ]:
main_df.shape


In [ ]:
# predicting df

data_20150602_1200_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150602_1200.csv")

### Feature Selection

##### Preliminary check

In [ ]:
main_df.shape

In [ ]:
data_20150602_1200_df.shape

In [ ]:
print(f"20191202:")
main_df.info()
print(f"Columns with NaN: {main_df[main_df.isna().any(axis=COL)]}")

print()

print(f"20191203:")
data_20150602_1200_df.info()
print(f"Columns with NaN: {data_20150602_1200_df[data_20150602_1200_df.isna().any(axis=COL)]}")

In [ ]:
print(f"20191202 - unique eqid count: {main_df.eqid.nunique()}")
print(f"20191203 - unique eqid count: {data_20150602_1200_df.eqid.nunique()}")

In [ ]:
main_df.drop('eqid', axis=COL, inplace=True)
data_20150602_1200_df.drop('eqid', axis=COL, inplace=True)

In [ ]:
print(f"training set:")
main_df.describe().loc[['min', 'max']]

In [ ]:
print(f"20150602:")
data_20150602_1200_df.describe().loc[['min', 'max']]

In [ ]:
RESPONSE_NAME

##### Get the responses columns

In [ ]:
training_responses = main_df[RESPONSE_NAME]
testing_responses = data_20150602_1200_df[RESPONSE_NAME]

##### 20191202_1200 data

In [ ]:
main_df.drop(RESPONSE_NAME, axis=COL, inplace=True)

In [ ]:
main_df.corr().isna().sum()

Some column pairs have NaN correlations, meaning either or both columns have constant values in all rows.

In [ ]:
# Get columns where all rows have the same value
constVal_columns = main_df.columns[main_df.apply(lambda x: x.nunique() == 1)]
constVal_columns

In [ ]:
TOBE_REMOVED_20191202_1200_COLUMNS = list(constVal_columns)

##### 20191203_1200 data

In [ ]:
data_20150602_1200_df.drop(RESPONSE_NAME, axis=COL, inplace=True)

In [ ]:
data_20150602_1200_df.corr().isna().sum()

In [ ]:
# Get columns where all rows have the same value
constVal_columns = data_20150602_1200_df.columns[data_20150602_1200_df.apply(lambda x: x.nunique() == 1)]
constVal_columns

In [ ]:
TOBE_REMOVED_20191203_1200_COLUMNS = list(constVal_columns)

##### Get training_predictors_df and testing_predictors_df

In [ ]:
TOBE_REMOVED_20191202_1200_COLUMNS_set = set(TOBE_REMOVED_20191202_1200_COLUMNS)
TOBE_REMOVED_20191203_1200_COLUMNS_set = set(TOBE_REMOVED_20191203_1200_COLUMNS)

In [ ]:
# Appear in TOBE_REMOVED_20191202_1200_COLUMNS_set but not in TOBE_REMOVED_20191203_1200_COLUMNS_set
print(TOBE_REMOVED_20191202_1200_COLUMNS_set - TOBE_REMOVED_20191203_1200_COLUMNS_set)

In [ ]:
# Appear in TOBE_REMOVED_20191203_1200_COLUMNS_set but not in TOBE_REMOVED_20191202_1200_COLUMNS_set
print(TOBE_REMOVED_20191203_1200_COLUMNS_set - TOBE_REMOVED_20191202_1200_COLUMNS_set)

In [ ]:
remove_20191202_stay_20191203_columns = list(TOBE_REMOVED_20191202_1200_COLUMNS_set - TOBE_REMOVED_20191203_1200_COLUMNS_set)
remove_20191202_stay_20191203_columns

In [ ]:
# data_20150602_1200_df[remove_20191202_stay_20191203_columns].describe()

In [ ]:
TOBE_REMOVED_COLUMNS = TOBE_REMOVED_20191202_1200_COLUMNS

Since there are variances among those columns, I don't think removing them now benefits the model.

In [ ]:
data_20191202_1200_working_df = main_df.drop(TOBE_REMOVED_COLUMNS, axis=COL)
data_20191203_1200_working_df = data_20150602_1200_df.drop(TOBE_REMOVED_COLUMNS, axis=COL)

In [ ]:
'rrirpnxm_nt_0' in main_df.columns

Rosy: we will not remove the columns right now from above, instead, we will just select the predictor columns we want to use

In [ ]:
trainig_working_df =  main_df[['rrirpnxm_nt_0','rrirpnxm_lst15_0','rrirpnxm_toxhr_0','rrirpnxm_lsthrx15_0']].copy()
testing_working_df = data_20150602_1200_df[['rrirpnxm_nt_0','rrirpnxm_lst15_0','rrirpnxm_toxhr_0','rrirpnxm_lsthrx15_0']].copy()


In [ ]:
print(f"1% negative correlations ranking: {sorted(trainig_working_df.corr().quantile(0.1), reverse=True)}")
print(f"99% negative correlations ranking: {sorted(trainig_working_df.corr().quantile(0.9), reverse=True)}")
trainig_working_df.corr().describe()

In [ ]:
# Box to do figure out which columns to use
training_predictors_df = data_20191202_1200_working_df.copy()
testing_predictors_df = data_20191203_1200_working_df.copy()

In [ ]:
sorted(training_predictors_df.quantile(0.25) / training_predictors_df.quantile(0.01), reverse=True)[:2]

In [ ]:
sorted(training_predictors_df.quantile(0.99) / training_predictors_df.quantile(0.75), reverse=True)[:2]

In [ ]:
plt.hist(training_responses, bins=100)

#### Detect influential points

I will use Mahalanobis distance, which computes distances from the data's centre and create an oval boundary.

In [ ]:
mean_vector = np.mean(training_predictors_df, axis=0)
cov_matrix = np.cov(training_predictors_df, rowvar=False)
inv_cov_matrix = np.linalg.inv(cov_matrix)

In [ ]:
mahalanobis_distances = []
for index, row in training_predictors_df.iterrows():
    mahalanobis_distance = scipy.spatial.distance.mahalanobis(row, mean_vector, inv_cov_matrix)
    mahalanobis_distances.append(mahalanobis_distance)

In [ ]:
training_predictors_df['mahalanobis_dist'] = mahalanobis_distances
training_predictors_df.head()

In [ ]:
#TODO: Need discussion
mahal_threshold = 3 # Typical 'mahal_threshold' is 3
outlier_significance_val = .001

In [ ]:
training_predictors_df['mahal_p_value'] = 1 - scipy.stats.chi2.cdf(training_predictors_df['mahalanobis_dist'], mahal_threshold)

In [ ]:
outlier_condition = (training_predictors_df['mahal_p_value'] < outlier_significance_val)

In [ ]:
nonOutlier_indices = training_predictors_df[~outlier_condition].index

In [ ]:
temp_col = 'response'
training_predictors_df[temp_col] = training_responses

training_predictors_df = training_predictors_df[~outlier_condition]

training_responses = training_predictors_df[temp_col]
training_predictors_df.drop(temp_col, axis=COL, inplace=True)

In [ ]:
training_predictors_df.drop(['mahalanobis_dist', 'mahal_p_value'], axis = COL, inplace=True)
training_predictors_df.reset_index(drop=True)
training_predictors_df.head()

#### Merge features and responses

In [ ]:
train_df = pd.concat([training_predictors_df, pd.DataFrame({consts.RESPONSE_NAME: training_responses})], 
                     axis=COL)
test_df = pd.concat([testing_predictors_df, pd.DataFrame({consts.RESPONSE_NAME: testing_responses})], 
                    axis=COL)

### Output

In [ ]:
out_path = '../data/cleaned_data/'

In [ ]:
out_train_filename = 'training_data.csv'
out_test_filename = 'testing_data.csv'

In [ ]:
train_df.to_csv(out_path + out_train_filename, index=False)
test_df.to_csv(out_path + out_test_filename, index=False)

In [ ]:
%pip freeze > dependencies.txt

In [ ]:
files = helper.get_file_names('20150101','20150601') # get filenames from before 6/1/2015
files

In [ ]:
dfs = [pd.read_csv(consts.DATA_PATH_2015 + f"/{f}") for f in files]

In [ ]:
dfs[0].head(5)

In [ ]:
dfs[-1].tail(5)

In [ ]:
main_df = pd.concat(dfs)
main_df

In [ ]:
main_df.shape

In [ ]:
test_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150601_1200.csv")

In [ ]:
test_df = pd.read_csv(consts.DATA_PATH_2015 + "data.20150601_1200.csv")
x_cols = ["rrirpnxm_nt_0", "rrirpnxm_lst15_0","rrirpnxm_lsthrx15_0", "rrirpnxm_toxhr_0"]
saved_cols = x_cols + [RESPONSE_NAME]
training_df = main_df[saved_cols]
testing_df = test_df[saved_cols]
